In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
test_data.head()

NameError: name 'test_data' is not defined

In [ ]:
train_data.columns

In [ ]:
from sklearn.model_selection import train_test_split

#Choosing features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

#Checking missing values
for i in features:
    print(i, " -> ", end="")
    print(f"No. of NaN values: {train_data[i].isna().sum()}")

In [ ]:
#Imputing the missing values
train_data.dropna(subset=['Embarked'], inplace=True)
train_data['Age'] = train_data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
# train_data['Age'] = train_data.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))

#Checking after imputation
for i in features:
    print(i, " -> ", end="")
    print(f"No. of NaN values: {train_data[i].isna().sum()}")

In [ ]:
#Splitting the data
y = train_data.Survived
#One-hot Encoding
X = pd.get_dummies(train_data[features])

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8)

In [ ]:
arr = [X_train, X_val, y_train, y_val]
for i in arr:
    print(i.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

model = RandomForestClassifier(random_state = 42)
model.fit(X_train, y_train)
y_preds_prob = model.predict_proba(X_val)
y_preds = model.predict(X_val)

loss = log_loss(y_val, y_preds_prob)
loss

In [ ]:
np.sum(y_val - y_preds)

In [ ]:
#Finetuning
best_n_estimators = 0
min_loss = float('inf')
for i in range(50, 1000, 50):
    temp = RandomForestClassifier(n_estimators=i, random_state = 42)
    temp.fit(X_train, y_train)
    y_preds_prob_temp = temp.predict_proba(X_val)
    loss = log_loss(y_val, y_preds_prob_temp)
    print(f"n_estimators = {i}, mae = {loss}")
    if loss < min_loss:
        min_loss = loss
        best_n_estimators = i
    
print(f"Best n_estimators = {best_n_estimators}, with log loss = {min_loss}")

In [ ]:
#Training the data ont the entire dataset
final_model = RandomForestClassifier(n_estimators = best_n_estimators, random_state = 42)
final_model.fit(X, y)
y_preds_prob = final_model.predict_proba(X_val)
y_preds = final_model.predict(X_val)

loss = log_loss(y_val, y_preds_prob)
loss

In [ ]:
#Making final predictions
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

#Checking for Missing data in test set
print("Before imputation")
for i in features:
    print(i, " -> ", end="")
    print(f"No. of NaN values: {test_data[i].isna().sum()}")
print("")

# #Imputing the test data
test_data['Age'] = test_data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)


#Checking after imputation
print("After imputation")
for i in features:
    print(i, " -> ", end="")
    print(f"No. of NaN values: {test_data[i].isna().sum()}")

#Making data to make predictions on
X_test = pd.get_dummies(test_data[features])
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
    
#Final preds
predictions = final_model.predict(X_test)

In [ ]:
#Submission
# output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
# output.to_csv('submission.csv', index=False)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv')